In [1]:
import pandas as pd
import os
from tqdm import tqdm
from multiprocessing import Pool
import numpy as np

In [2]:
def flatten(lst):
    return [item for sublist in lst if type(sublist)==list for item in sublist  ]
    

In [3]:
chart_res_df = pd.concat([pd.read_csv('data/all_admission_fulladm_chart_stats_merged.csv',index_col=0), pd.read_csv('data/all_admission_fulladm_lab_stats_merged.csv',index_col=0)], ignore_index=True)
chart_res_df = chart_res_df.fillna('[]')
chart_res_df[list(chart_res_df.columns)[1:]] = chart_res_df[list(chart_res_df.columns)[1:]].applymap(eval)

# for col in chart_res_df.columns[1:]:
#     chart_res_df[col]=chart_res_df[col].apply(eval)

/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
med_df = pd.read_csv('../../LHC_mimic/mimic3_1.4/raw_data/PRESCRIPTIONS.csv')

/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
chart_res_df = pd.concat([chart_res_df.groupby('new_HADM').agg(lambda x: flatten(list(x)))[list(chart_res_df.columns)[1:]].applymap(lambda x: np.mean(x) ).rename(lambda x: 'mean_'+x, axis=1),
chart_res_df.groupby('new_HADM').agg(lambda x: flatten(list(x)))[list(chart_res_df.columns)[1:]].applymap(lambda x: min(x,default=np.nan) ).rename(lambda x: 'min_'+x, axis=1),
chart_res_df.groupby('new_HADM').agg(lambda x: flatten(list(x)))[list(chart_res_df.columns)[1:]].applymap(lambda x: max(x,default=np.nan) ).rename(lambda x: 'max_'+x, axis=1),
chart_res_df.groupby('new_HADM').agg(lambda x: flatten(list(x)))[list(chart_res_df.columns)[1:]].applymap(lambda x: pd.Series(x).count() ).rename(lambda x: 'count_'+x, axis=1),
chart_res_df.groupby('new_HADM').agg(lambda x: flatten(list(x)))[list(chart_res_df.columns)[1:]].applymap(lambda x: np.median(x) ).rename(lambda x: 'median_'+x, axis=1),
chart_res_df.groupby('new_HADM').agg(lambda x: flatten(list(x)))[list(chart_res_df.columns)[1:]].applymap(lambda x: np.std(x) ).rename(lambda x: 'stdev_'+x, axis=1)], axis=1)

/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.
/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/gpfs/gsfs12/users/moldwinar/environments/torchtcav/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid val

In [6]:
chart_res_df

,mean_cholesterol,mean_diasbp,mean_glucose,mean_hdl,mean_heartrate,mean_ldl,mean_meanbp,mean_resprate,mean_spo2,mean_sysbp,...,stdev_meanbp,stdev_resprate,stdev_spo2,stdev_sysbp,stdev_tempc,stdev_triglycerides,stdev_abd_girth,stdev_heightcm,stdev_weightkg,stdev_hema1c
new_HADM,,,,,,,,,,,,,,,,,,,,,
100001.0,NaN,92.780822,177.000000,NaN,111.313953,NaN,111.852941,17.357143,97.604167,174.191781,...,13.065744,2.467131,0.951744,21.355861,0.454595,NaN,NaN,NaN,NaN,NaN
100003.0,NaN,60.615385,95.300000,NaN,81.218182,NaN,73.980769,15.818182,98.229167,116.519231,...,11.930069,2.790184,2.717532,16.891612,0.499362,NaN,NaN,NaN,NaN,NaN
100006.0,NaN,82.169811,140.342857,NaN,108.630252,NaN,101.616366,18.682927,97.861789,140.509434,...,11.199060,3.308813,2.554528,12.911342,0.346422,NaN,NaN,NaN,0.000000,NaN
100007.0,NaN,67.164557,124.333333,NaN,79.444444,NaN,91.000000,19.185185,97.960526,140.746835,...,10.330889,4.433629,2.173045,12.225175,0.674540,NaN,NaN,NaN,0.550001,NaN
100009.0,NaN,51.175439,154.307692,NaN,68.968750,NaN,66.178571,23.888889,96.174603,110.543860,...,7.438095,5.671178,2.012183,12.540559,0.280822,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199993.0,NaN,66.012755,122.576119,25.0,87.219136,111.0,80.944870,18.705634,96.533003,106.927296,...,12.273041,4.616791,1.907446,14.164614,0.474080,NaN,NaN,0.0,7.162080,NaN
199994.0,NaN,51.043062,114.960000,NaN,71.882353,NaN,70.645730,13.833333,96.948454,107.695238,...,14.008126,3.673479,2.560087,16.348947,0.595796,NaN,NaN,0.0,7.317915,NaN
199995.0,NaN,63.169811,115.378788,NaN,108.150943,NaN,79.477983,18.716981,97.367347,112.830189,...,8.309130,4.435608,2.413346,10.860585,0.533723,NaN,NaN,0.0,0.000000,0.0


In [7]:
rx_df = pd.read_csv('data/all_admission_fulladm_rx_stats_merged.csv', index_col=0)
rx_df = rx_df.groupby('new_HADM').agg(lambda x: list(x))[list(rx_df.columns)[1:]].applymap(lambda x: sum(x)>0 )

In [8]:
dx_df = pd.read_csv('data/all_admission_fulladm_dx_stats_merged.csv', index_col=0)
dx_df = dx_df.drop_duplicates(subset=['new_HADM'])#.groupby('HADM_ID').agg(lambda x: list(x))# [list(dx_df.columns)[1:]].applymap(lambda x: sum(x)>0 )

In [9]:
chart_res_df =chart_res_df.reset_index()
chart_res_df['new_HADM'] = chart_res_df['new_HADM'].astype(int)

In [10]:
rx_df= rx_df.reset_index()
rx_df['new_HADM'] = rx_df['new_HADM'].astype(int)

In [11]:
# dx_df= dx_df.reset_index()
dx_df['new_HADM'] = dx_df['new_HADM'].astype(int)

In [12]:
dx_df

,new_HADM,diabetesdx
71,140784,1
91,164853,1
113,195632,1
206,113323,1
214,198214,1
...,...,...
650799,138027,1
650811,149629,1
650880,109999,1
650990,152542,1


In [13]:
all_df = chart_res_df.merge( rx_df, on='new_HADM', how='outer').merge( dx_df, on='new_HADM', how='outer')

In [14]:
all_df

,new_HADM,mean_cholesterol,mean_diasbp,mean_glucose,mean_hdl,mean_heartrate,mean_ldl,mean_meanbp,mean_resprate,mean_spo2,...,stdev_abd_girth,stdev_heightcm,stdev_weightkg,stdev_hema1c,MED_ACE,MED_BETA,MED_FIBR,MED_INS,MED_STAT,diabetesdx
0,100001,NaN,92.780822,177.000000,NaN,111.313953,NaN,111.852941,17.357143,97.604167,...,NaN,NaN,NaN,NaN,False,False,False,True,True,1.0
1,100003,NaN,60.615385,95.300000,NaN,81.218182,NaN,73.980769,15.818182,98.229167,...,NaN,NaN,NaN,NaN,False,False,False,False,True,NaN
2,100006,NaN,82.169811,140.342857,NaN,108.630252,NaN,101.616366,18.682927,97.861789,...,NaN,NaN,0.000000,NaN,False,False,False,True,True,NaN
3,100007,NaN,67.164557,124.333333,NaN,79.444444,NaN,91.000000,19.185185,97.960526,...,NaN,NaN,0.550001,NaN,True,False,False,True,True,NaN
4,100009,NaN,51.175439,154.307692,NaN,68.968750,NaN,66.178571,23.888889,96.174603,...,NaN,NaN,NaN,0.0,True,False,True,True,True,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57685,100580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
57686,101556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
57687,132498,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
57688,115270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [16]:
all_df.to_csv('data/all_admission_fulladm_all_stats_merged.csv')